# EMD for extracting features

Using Empirical Mode Decomposition to extract features for data series. For each of the bearings, extract the desired number of EMDs and then compute descriptive statistics of them.

Documentation links:

[Empirical Mode Decomposition in Python](https://emd.readthedocs.io/en/stable/index.html)

[The empirical mode decomposition and the Hilbert spectrum for nonlinear and non-stationary time series analysis](https://royalsocietypublishing.org/doi/epdf/10.1098/rspa.1998.0193)

## Imports

In [ ]:
import scipy.io
import os
import pandas as pd
import numpy as np

import emd

## Functions

In [ ]:
# create dictionaries to map the file name to the operating conditions => not gonna be used in the classification but kept for reference
'''https://mb.uni-paderborn.de/en/kat/main-research/datacenter/bearing-datacenter/operating-conditions'''

dict_rotation_speed = {'N15':1500,'N09':900}
dict_load_torque = {'M07':0.7, 'M01':0.1}
dict_radial_force = {'F10':1000,'F04':400}

# map_operating_conditions
# input: pandas dataframe, file_name
# output: dataframe with columns with the numerical values of rotation speed, load torque and radial force
def map_operating_conditions(file_name):
    rotation_speed = dict_rotation_speed[file_name[:3]]
    load_torque = dict_load_torque[file_name[4:7]]
    radial_force = dict_radial_force[file_name[8:11]]
    bearing = file_name[12:16]

    return rotation_speed, load_torque, radial_force, bearing

# extract_measures
# input: dataset with measures
# output: arrays of phase current 1, phase current 2 and vibration

def extract_measures(data):

  # extract arrays with each information => for now, going to focus on current and vibration
  phase_current_1 = data[0][0][0][1][2][0]
  phase_current_2 = data[0][0][0][2][2][0]
  vibration = data[0][0][0][6][2][0]

  return phase_current_1, phase_current_2, vibration

# get_descriptive_measures
# input: dataframe to save information, array to compute descriptive measures
# output: dataframe with min, max, median, mean, rms, std and range of the array

def get_descriptive_measures(arrays, names):
    data = {}

    for array, name in zip(arrays, names):
        data[f'min_{name}'] = np.min(array)
        data[f'max_{name}'] = np.max(array)
        data[f'median_{name}'] = np.median(array)
        data[f'mean_{name}'] = np.mean(array)
        data[f'rms_{name}'] = np.sqrt(np.vdot(array, array)/array.size)
        data[f'std_{name}'] = np.std(array)
        data[f'range_{name}'] = np.ptp(array)

    df = pd.DataFrame(data,index=[0])
    return df

# apply EMD
# input:
# n_imfs - number of imfs to decompose
# array - data to be decomposed
# output
# decomposed series, n_imfs arrays of format format (1, len(array))

def apply_EMD(n_imfs,array):

  # get imfs
  imfs = emd.sift.mask_sift(array, max_imfs=n_imfs)

  arrays_emd = []

  for i in range(n_imfs):
    arrays_emd.append(imfs[:,i])

  return arrays_emd

## EMD parameters

In [ ]:
n_imf = 5

## Undamaged Bearings

In [ ]:
directory_path = "C:\\Users\\julia\\Documents\\UNICAMP\\TCC\\KAT\\"
normal_bearings = ['K001','K002','K003','K004','K005','K006']

# for each of the undamaged bearings get the list of files associated with it
for bearing in normal_bearings:
  files = os.listdir(directory_path+f'{bearing}')
  print(bearing)
  dataframes = []

  for i in range(len(files)):
    measure_file = files[i]
    if (measure_file[-3:]=='mat'): #check if it is a .mat file
      print(measure_file)

      # read file
      data = scipy.io.loadmat(f"C:\\Users\\julia\\Documents\\UNICAMP\\TCC\\KAT\\{bearing}\\{measure_file}",appendmat=False)[f'{measure_file[:-4]}']['Y']

      # get operating conditions
      rotation_speed, load_torque, radial_force, bearing = map_operating_conditions(measure_file)

      # get array with measures
      phase_current_1, phase_current_2, vibration = extract_measures(data)

      # split each of the 4s series in 4 series of 1s
      start = 0
      end = len(phase_current_1)//4

      for s in range(4):

        # aux vectors
        measures = []
        names = []

        # extract 5 components from the series using EMD
        phase_current_1_emd = apply_EMD(n_imf, phase_current_1[start:end])
        phase_current_2_emd = apply_EMD(n_imf, phase_current_2[start:end])
        vibration_emd = apply_EMD(n_imf, vibration[start:end])

        # get metrics for each decomposed serie
        for j in range(n_imf):
          measures.append(phase_current_1_emd[j])
          measures.append(phase_current_2_emd[j])
          measures.append(vibration_emd[j])

          names.append(f"phase_current_1_emd_{j}")
          names.append(f"phase_current_2_emd_{j}")
          names.append(f"vibration_emd_{j}")

        aux_df = pd.DataFrame()
        aux_df = get_descriptive_measures(measures,names)

        aux_df['rotation_speed'] = rotation_speed
        aux_df['load_torque'] = load_torque
        aux_df['radial_force'] = radial_force
        aux_df['bearing'] = bearing
        aux_df['label'] = 0
        aux_df['label_detailed'] = 'normal'

        dataframes.append(aux_df)

        start += len(phase_current_1)//4
        end += len(phase_current_1)//4

  bearing_info = pd.concat(dataframes, ignore_index=True)
  bearing_info.to_csv(f"C:\\Users\\julia\\Documents\\UNICAMP\\TCC\\Processed_KAT\\EMD_n10\\{bearing}_emd_n5.csv", index=False)

## Accelerated lifetime

In [ ]:
directory_path = "C:\\Users\\julia\\Documents\\UNICAMP\\TCC\\KAT\\"
acc_lifetime_bearings = ['KA04', 'KA15', 'KA16', 'KA22', 'KA30', 'KB23', 'KB24', 'KB27', 'KI04', 'KI14', 'KI16', 'KI17', 'KI18', 'KI21']

# for each of the undamaged bearings get the list of files associated with it
for bearing in acc_lifetime_bearings:
  files = os.listdir(directory_path+f'{bearing}')
  print(bearing)
  dataframes = []

  for i in range(len(files)):
    measure_file = files[i]
    if (measure_file[-3:]=='mat'): #check if it is a .mat file
      print(measure_file)

      # read file
      data = scipy.io.loadmat(f"C:\\Users\\julia\\Documents\\UNICAMP\\TCC\\KAT\\{bearing}\\{measure_file}",appendmat=False)[f'{measure_file[:-4]}']['Y']

      # get operating conditions
      rotation_speed, load_torque, radial_force, bearing = map_operating_conditions(measure_file)

      # get array with measures
      phase_current_1, phase_current_2, vibration = extract_measures(data)

      # split each of the 4s series in 4 series of 1s
      start = 0
      end = len(phase_current_1)//4

      for s in range(4):

        # aux vectors
        measures = []
        names = []

        # extract 5 components from the series using EMD
        phase_current_1_emd = apply_EMD(n_imf, phase_current_1[start:end])
        phase_current_2_emd = apply_EMD(n_imf, phase_current_2[start:end])
        vibration_emd = apply_EMD(n_imf, vibration[start:end])

        # get metrics for each decomposed serie
        for j in range(n_imf):
          measures.append(phase_current_1_emd[j])
          measures.append(phase_current_2_emd[j])
          measures.append(vibration_emd[j])

          names.append(f"phase_current_1_emd_{j}")
          names.append(f"phase_current_2_emd_{j}")
          names.append(f"vibration_emd_{j}")

        aux_df = pd.DataFrame()
        aux_df = get_descriptive_measures(measures,names)

        aux_df['rotation_speed'] = rotation_speed
        aux_df['load_torque'] = load_torque
        aux_df['radial_force'] = radial_force
        aux_df['bearing'] = bearing
        aux_df['label'] = 0
        aux_df['label_detailed'] = 'normal'

        dataframes.append(aux_df)

        start += len(phase_current_1)//4
        end += len(phase_current_1)//4

  bearing_info = pd.concat(dataframes, ignore_index=True)
  bearing_info.to_csv(f"C:\\Users\\julia\\Documents\\UNICAMP\\TCC\\Processed_KAT\\EMD_n5\\{bearing}_emd_n5.csv", index=False)

## Artificially damaged

In [ ]:
directory_path = "C:\\Users\\julia\\Documents\\UNICAMP\\TCC\\KAT\\"
artificial_damaged_bearings =['KA01','KA03','KA05','KA06','KA07','KA08','KA09','KI01','KI03','KI05','KI07','KI08']

# for each of the undamaged bearings get the list of files associated with it
for bearing in artificial_damaged_bearings:
  files = os.listdir(directory_path+f'{bearing}')
  print(bearing)
  dataframes = []

  for i in range(len(files)):
    measure_file = files[i]
    if ((measure_file[-3:]=='mat')&(measure_file !="N15_M01_F10_KA08_2.mat")): #check if it is a .mat file
      print(measure_file)

      # read file
      data = scipy.io.loadmat(f"C:\\Users\\julia\\Documents\\UNICAMP\\TCC\\KAT\\{bearing}\\{measure_file}",appendmat=False)[f'{measure_file[:-4]}']['Y']

      # get operating conditions
      rotation_speed, load_torque, radial_force, bearing = map_operating_conditions(measure_file)

      # get array with measures
      phase_current_1, phase_current_2, vibration = extract_measures(data)

      # split each of the 4s series in 4 series of 1s
      start = 0
      end = len(phase_current_1)//4

      for s in range(4):

        # aux vectors
        measures = []
        names = []

        # extract 5 components from the series using EMD
        phase_current_1_emd = apply_EMD(n_imf, phase_current_1[start:end])
        phase_current_2_emd = apply_EMD(n_imf, phase_current_2[start:end])
        vibration_emd = apply_EMD(n_imf, vibration[start:end])

        # get metrics for each decomposed serie
        for j in range(n_imf):
          measures.append(phase_current_1_emd[j])
          measures.append(phase_current_2_emd[j])
          measures.append(vibration_emd[j])

          names.append(f"phase_current_1_emd_{j}")
          names.append(f"phase_current_2_emd_{j}")
          names.append(f"vibration_emd_{j}")

        aux_df = pd.DataFrame()
        aux_df = get_descriptive_measures(measures,names)

        aux_df['rotation_speed'] = rotation_speed
        aux_df['load_torque'] = load_torque
        aux_df['radial_force'] = radial_force
        aux_df['bearing'] = bearing
        aux_df['label'] = 0
        aux_df['label_detailed'] = 'normal'

        dataframes.append(aux_df)

        start += len(phase_current_1)//4
        end += len(phase_current_1)//4


  bearing_info = pd.concat(dataframes, ignore_index=True)
  bearing_info.to_csv(f"C:\\Users\\julia\\Documents\\UNICAMP\\TCC\\Processed_KAT\\EMD_n5\\{bearing}_emd_n5.csv", index=False)